# Conic Solver Approach for Implied Returns

This notebook reformulates the Nelder-Mead optimization approach from notebook 10 using cvxpy conic solvers.

## DEPENDENCIES

In [1]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize
import cvxpy as cp 

## LOAD INPUT PRICE FILE

In [2]:
def load_source(sourcefile):
    try:
        source_df=pd.read_csv(sourcefile)
        temp=source_df.get('Date')
        if not temp is None:
            source_df.index=temp 
            source_df=source_df.drop(columns=['Date'])
        return source_df
    except:
        print('NO SOURCE FOUND')
        return None

In [3]:
def implied_dif_returns_conic(rtns, lev, allocation, worst, target_exputil):
    nrows, ncols = rtns.shape
    
    # Decision variable: adjustment to returns
    x = cp.Variable(ncols)
    
    # Auxiliary variables for log terms
    t = cp.Variable(nrows)
    
    # Calculate adjusted portfolio returns
    # rtns is (nrows, ncols), x is (ncols,), allocation is (ncols,1)
    # We need to broadcast x to match rtns shape
    adjusted_rtns = rtns + cp.reshape(x, (1, ncols), order='F')
    port_rtns = adjusted_rtns @ allocation  # This results in (nrows, 1)
    lev_port_rtns = lev * port_rtns
    
    # Constraints
    constraints = []
    
    # Ensure 1 + lev_port_rtns > 0 (needed for log1p)
    constraints.append(1 + lev_port_rtns >= 1e-8)
    
    # Worst return constraint
    constraints.append(lev_port_rtns >= worst)
    
    # Log constraint using exponential cone
    # We want: t[i] <= log(1 + lev_port_rtns[i])
    # This is equivalent to: exp(t[i]) <= 1 + lev_port_rtns[i]
    for i in range(nrows):
        # ExpCone requires all arguments to be scalars
        # lev_port_rtns has shape (nrows, 1), so we use [i, 0] to get scalar
        constraints.append(cp.constraints.ExpCone(t[i], 1.0, (1 + lev_port_rtns)[i, 0]))
    
    # Target expected utility constraint
    # We want: sum(t) / nrows / lev = target_exputil
    constraints.append(cp.sum(t) == target_exputil * nrows * lev)
    
    # Objective: minimize norm of adjustments
    objective = cp.Minimize(cp.norm(x, 2))
    
    prob = cp.Problem(objective, constraints)
    
    try:
        result = prob.solve(solver=cp.CLARABEL, verbose=False)
        if x.value is None:
            print("Warning: cvxpy problem appears not feasible.")
            # Fall back to Nelder-Mead
            return implied_dif_returns_nelder(rtns, lev, allocation, worst, target_exputil)
        return x.value
    except Exception as e:
        print(f"Conic solver failed: {e}")
        # Fall back to Nelder-Mead
        return implied_dif_returns_nelder(rtns, lev, allocation, worst, target_exputil)

In [4]:
def calculate_returns(source_df):
    price_data=np.array(source_df.values,dtype='float64')
    price_data1=np.ones((price_data.shape[0],price_data.shape[1]))
    price_data1[1:]=price_data[:-1]
    returns=(price_data/price_data1)
    returns=returns[1:]-1. 
    returns_df=pd.DataFrame(returns,columns=source_df.columns,index=source_df.index[1:])   
    return(returns_df)

## FIND BEST ALLOCATION

In [5]:
def find_best_allocation(rtns_df,lev,long_only,worst):
    rtns=rtns_df.values
    nrows,ncols=rtns.shape
    levreturn=(rtns*lev)
    
    xx=cp.Variable(ncols)
    if long_only:
        constraints =[sum(xx)==1, 0<=xx, xx<=1, worst <= levreturn @ xx ]
    else:
        constraints = [sum(xx)==1,worst <= levreturn @ xx ]
    objective=cp.Minimize(cp.sum(-cp.log1p(levreturn @ xx)))
    prob=cp.Problem(objective,constraints)
    result=prob.solve(solver=cp.CLARABEL,tol_feas=1e-7,tol_gap_abs=1e-7, tol_gap_rel=1e-7, tol_ktratio=1e-7, verbose=False) /nrows/lev
    xxvalue=xx.value #allocation
            
    if xxvalue is None:                
        print('WARNING!!!! cvxpy problem mappears not feasible.')
        return None
                
    prtns=np.dot(rtns,xxvalue)     
    alloc=xxvalue 

    return ('dummy',prtns,xxvalue,-result)

## IMPLIED RETURNS ESTIMATOR - CONIC SOLVER VERSION

This reformulates the Nelder-Mead approach using cvxpy with conic constraints.
The key insight is to introduce auxiliary variables to handle the log terms.

## IMPLIED RETURNS ESTIMATOR - NELDER-MEAD (FALLBACK)

In [6]:
def implied_dif_returns_nelder(rtns, lev, allocation, worst, target_exputil):

    nrows, ncols = rtns.shape
    
    def objective(x):
        adjusted_rtns = rtns + x[np.newaxis, :]
        port_rtns = adjusted_rtns @ allocation
        lev_port_rtns = lev * port_rtns
        
        if np.any(1 + lev_port_rtns <= 0):
            return 1e10
        
        exp_util = np.sum(np.log1p(lev_port_rtns)) / nrows / lev
        return (exp_util - target_exputil)**2
    
    x0 = np.zeros(ncols)
    result = minimize(objective, x0, method='Nelder-Mead', 
                     options={'xatol': 1e-8, 'fatol': 1e-12, 'maxiter': 20000})
    
    return result.x

## IMPLIED EXPECTED RETURN

In [7]:
def find_implied_dif_expected_returns(returns_df, lev, worst, walloc, exputil, use_conic=True):
    rtns = returns_df.values
    ncols=rtns.shape[1]
    
    #allocation = walloc  # This allocation seems likely to produce differential returns near zero,
    #since it was the optimum given the same context.
    allocation =(1./ncols) * np.ones((ncols,1))  #This one is of greater interest
    
    if use_conic:
        output = implied_dif_returns_conic(rtns, lev, allocation, worst, exputil).T
    else:
        output = implied_dif_returns_nelder(rtns, lev, allocation, worst, exputil).T
   
    return output

## PRINT PARAMETERS

In [8]:
def print_parameters(sourcefile,sourcetype,Llist,long_only,worst,actual_alloc,expected_returns):
    print(' ')    
    print(f'{sourcefile=}')
    print(f'{sourcetype=}')
    print(f'{Llist=}')
    print(f'{long_only=}') 
    print(f'{worst=}')
    print(f'{actual_alloc=}')
    print(f'{expected_returns=}')
    print(' ')
    return    

## MAIN PROGRAM

In [9]:
def woptimize(params={}):

    sourcefile=params.get('sourcefile')
    sourcetype=params.get('sourcetype')    
    Llist=params.get('Llist')
    long_only=params.get('long_only')
    worst=params.get('worst')
    actual_alloc=params.get('actual_alloc')
    expected_returns=params.get('expected_returns')
    use_conic=params.get('use_conic', True)
    
    #record control parameters
    print_parameters(sourcefile,sourcetype,Llist,long_only,worst,actual_alloc, expected_returns)
        
    #Read in Prices or Returns, based on sourcetype, adjusted for dividends and interest if possible
    if sourcetype=='PRICES':        
        #Calculate return matrix
        returns_df=calculate_returns(load_source(sourcefile))
    elif sourcetype=='RETURNS':
        returns_df=load_source(sourcefile)
    else:
        print('UNABLE TO DETERMINE SOURCE TYPE')
        raise
    print(returns_df.head())
    
    #log leveraged surplus optimizations
    big_exputil_df=pd.DataFrame(np.zeros((1,len(Llist))),columns=Llist)
    big_walloc=np.zeros((len(returns_df.columns),len(Llist)))
    big_walloc_df = pd.DataFrame(big_walloc,columns=Llist,index=returns_df.columns)
    big_implied_dif = np.zeros((len(returns_df.columns),len(Llist)))
    big_implied_dif_df = pd.DataFrame(big_implied_dif,columns=Llist,index=returns_df.columns)
    for lev in Llist:
        (error_code1, wpreturns,walloc,exputil) = find_best_allocation(returns_df,lev,long_only,worst)
        big_walloc_df[lev]=walloc
        big_exputil_df[lev]=exputil
        big_implied_dif_df[lev] = find_implied_dif_expected_returns(returns_df,lev,worst,walloc,exputil,use_conic)
         
    with pd.option_context('display.float_format', '{:,.5f}'.format):
        print(' ')
        print('OPTIMAL ALLOCATIONS')
        print(big_walloc_df)
        print(' ')
        print('EXPECTED UTILITIES')
        print(big_exputil_df)
        print(' ')
        print('IMPLIED DIFFERENCE IN EXPECTED RETURN (CONIC SOLVER)' if use_conic else 'IMPLIED DIFFERENCE IN EXPECTED RETURN (NELDER-MEAD)')
        print(big_implied_dif_df)
    print(' ')        
    
    print('DONE!')
    
    return

## SET PARAMETERS AND RUN OPTIMIZATION WITH CONIC SOLVER

In [10]:
#set parameters
params=dict(
    sourcefile='DATA20/prices.csv',
    sourcetype='PRICES',
    Llist=[1,2,4,8],
    long_only=True,
    worst=(-0.99),
    actual_alloc=None,
    expected_returns=None,
    use_conic=True  # Use conic solver
    )

#run main program with conic solver
print("Running with CONIC SOLVER:")
print("="*50)
optimizer_output=woptimize(params)

Running with CONIC SOLVER:
 
sourcefile='DATA20/prices.csv'
sourcetype='PRICES'
Llist=[1, 2, 4, 8]
long_only=True
worst=-0.99
actual_alloc=None
expected_returns=None
 
               VWSTX       XLE       EWJ     VWEHX       XLP     VWAHX  \
Date                                                                     
1999-01-01  0.004880 -0.065596  0.025385  0.019328 -0.013233  0.018370   
1999-02-01  0.001924 -0.008597 -0.035715 -0.005854 -0.010496 -0.002812   
1999-03-01  0.000833  0.137284  0.135803  0.007603 -0.002947 -0.000547   
1999-04-01  0.002508  0.152157  0.043478  0.013409 -0.033165  0.004415   
1999-05-01  0.001801 -0.021583 -0.057292 -0.016409 -0.010410 -0.006809   

               VFIIX       XLV     VUSTX     VWESX       XLY       DIA  \
Date                                                                     
1999-01-01  0.008281  0.048077  0.009904  0.028579  0.051435  0.020846   
1999-02-01 -0.007106  0.001147 -0.047315 -0.036633 -0.006257 -0.002542   
1999-03-01  0.006

## COMPARE WITH NELDER-MEAD

In [11]:
#run with Nelder-Mead for comparison
params['use_conic'] = False
print("\nRunning with NELDER-MEAD for comparison:")
print("="*50)
optimizer_output_nm=woptimize(params)


Running with NELDER-MEAD for comparison:
 
sourcefile='DATA20/prices.csv'
sourcetype='PRICES'
Llist=[1, 2, 4, 8]
long_only=True
worst=-0.99
actual_alloc=None
expected_returns=None
 
               VWSTX       XLE       EWJ     VWEHX       XLP     VWAHX  \
Date                                                                     
1999-01-01  0.004880 -0.065596  0.025385  0.019328 -0.013233  0.018370   
1999-02-01  0.001924 -0.008597 -0.035715 -0.005854 -0.010496 -0.002812   
1999-03-01  0.000833  0.137284  0.135803  0.007603 -0.002947 -0.000547   
1999-04-01  0.002508  0.152157  0.043478  0.013409 -0.033165  0.004415   
1999-05-01  0.001801 -0.021583 -0.057292 -0.016409 -0.010410 -0.006809   

               VFIIX       XLV     VUSTX     VWESX       XLY       DIA  \
Date                                                                     
1999-01-01  0.008281  0.048077  0.009904  0.028579  0.051435  0.020846   
1999-02-01 -0.007106  0.001147 -0.047315 -0.036633 -0.006257 -0.002542   
19